### Assignment 5 - Hamed Ahmadinia

![image info](https://www.mdpi.com/sensors/sensors-17-01951/article_deploy/html/images/sensors-17-01951-g003.png)


>SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and <0.5MB model size
(https://arxiv.org/abs/1602.07360)

In this assigment you're going to use the pretrained network SqueezeNetv1.2 (~ 5 Mb) 

#### Task 1 (5 points):

go to https://github.com/miaow1988/SqueezeNet_v1.2 and download the 'symbol.json' and '.params' files (there is not a 'synset.txt' file! so don't use these lines, Hint: just comment these lines).

* Install MXNet v1.5 (hint: create a new conda environmet with python 3, pip install mxnet==1.5.1) and follow the same steps of the lecture (part: *Using pre-trained models as feature extractors*). Find the flatten output layer and create a feature extractor (hint: It should be a numpy array of 1000 elements).
* Download the dogs versus cats *training folder* from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data (Remember the number of images is 12500 for each class).
* Extract the array of features for different number of images (N: 10, 100, 500, 1000, also 5000 and 12500) and for each value train your favorite binary classifier (only one!!!) using GridSearch to optimize some hyperparameters. Consider to use https://notebooks.csc.fi if you have computational limitations. 

* Report the accuracy for each value of N and the computational time during the training step.

In [1]:
# To install mxnet 1.5.1
!pip install mxnet==1.5.1

In [2]:
# To install opencv_python_headless
!pip install opencv_python_headless

In [3]:
# Loading SqueezeNet_v1.2 
import mxnet as mx
path='https://github.com/miaow1988/SqueezeNet_v1.2/'
[mx.test_utils.download(path+'raw/master/model-symbol.json'),
 mx.test_utils.download(path+'raw/master/model-0000.params')]

['model-symbol.json', 'model-0000.params']

In [4]:
with open('model-symbol.json', 'r') as f:
    for n in range(10):
        s = f.readline().strip()
        if s: print(s)

{
"nodes": [
{
"op": "null",
"name": "data",
"inputs": []
},
{
"op": "null",
"name": "conv1_conv_weight",


In [5]:
sym, arg_params, aux_params = mx.model.load_checkpoint('model', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

In [6]:
# preparing matplotlib and importing cv2 and numpy
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
# defining a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

def get_image(url, show=False):
    # download and show the image
    fname = mx.test_utils.download(url, fname=url.split('/')[-1].split('?')[0])
    img = mx.image.imread(fname)
    if img is None:
         return None
    if show:
         plt.imshow(img.asnumpy())
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = mx.image.imagesize(img, (224, 224))
    img = img.transpose(2, 0, 1)
    img = img.expand_dims(axis=0)
    img = img.astype('float32')
    return img

def get_image(url, show=False):
    if url.startswith('http'):
        # download and show the image
        fname = mx.test_utils.download(url)
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict(url):
    img = get_image(url, show=True)
    # compute the predict probabilities
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    for i in a[0:5]:
        print('probability=%f, class=%s' %(prob[i], labels[i]))

In [7]:
# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['fire9_concat_output',
 'dropout0_output',
 'conv10_conv_weight',
 'conv10_conv_bias',
 'conv10_conv_output',
 'conv10_relu_output',
 'pool10_output',
 'flatten0_output',
 'softmax_label',
 'softmax_output']

In [8]:
fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [9]:
img = get_image('https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12234558/Chinook-On-White-03.jpg')
fe_mod.forward(Batch([mx.nd.array(img)]))
features = fe_mod.get_outputs()[0].asnumpy()
print(features)
assert features.shape == (1, 1000)

[[ 3.8680282   3.0770736   3.8007517   3.9388447   4.4794497   2.513034
   1.0257548   3.3912466   4.1260457   1.1125107   3.0114415   2.2824717
   3.308097    3.6617973   2.2812922   1.8932449   4.174476    3.710502
   0.45999247  4.1961174   1.2289764   9.014702    6.0741653   5.186479
   4.293989    3.2768373   4.330217    3.7503705   6.209451    4.7506633
   3.809687    8.039387    5.815378    4.479296    4.644457    2.287041
   4.105688    3.3356824   8.155916    4.587722    4.465425    3.855896
   4.043045    5.1172795   4.4702134   3.342398    5.4953494   6.7198224
   3.84865     1.182495    2.0083091   9.924515    2.7754364   2.005786
   3.0824454   1.4339454   2.6161735   1.7753203   1.2789447   2.4843435
   1.5804738   6.9447136   4.049789    2.9182622   1.8067827   1.929847
   2.0441859   2.8852274   2.3605494   5.1194777   2.5484276   5.3415647
   3.779574    3.719067    4.224366    4.0597167   2.472064    2.177067
   6.892461    4.2934427   4.110542    5.4240932   6.028346

In [10]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [11]:
img = get_image('https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12234558/Chinook-On-White-03.jpg')
features = get_features(img)
print("{}\n shape: {}".format(features,features.shape))

[[ 3.8680282   3.0770736   3.8007517   3.9388447   4.4794497   2.513034
   1.0257548   3.3912466   4.1260457   1.1125107   3.0114415   2.2824717
   3.308097    3.6617973   2.2812922   1.8932449   4.174476    3.710502
   0.45999247  4.1961174   1.2289764   9.014702    6.0741653   5.186479
   4.293989    3.2768373   4.330217    3.7503705   6.209451    4.7506633
   3.809687    8.039387    5.815378    4.479296    4.644457    2.287041
   4.105688    3.3356824   8.155916    4.587722    4.465425    3.855896
   4.043045    5.1172795   4.4702134   3.342398    5.4953494   6.7198224
   3.84865     1.182495    2.0083091   9.924515    2.7754364   2.005786
   3.0824454   1.4339454   2.6161735   1.7753203   1.2789447   2.4843435
   1.5804738   6.9447136   4.049789    2.9182622   1.8067827   1.929847
   2.0441859   2.8852274   2.3605494   5.1194777   2.5484276   5.3415647
   3.779574    3.719067    4.224366    4.0597167   2.472064    2.177067
   6.892461    4.2934427   4.110542    5.4240932   6.028346

In [12]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "Dogs vs. Cats_train.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
train/                                         2013-09-20 10:12:24            0
train/cat.0.jpg                                2013-09-20 10:05:42        12414
train/cat.1.jpg                                2013-09-20 10:05:42        16880
train/cat.10.jpg                               2013-09-20 10:05:42        34315
train/cat.100.jpg                              2013-09-20 10:05:42        28377
train/cat.1000.jpg                             2013-09-20 10:05:42         5944
train/cat.10000.jpg                            2013-09-20 10:05:42        21944
train/cat.10001.jpg                            2013-09-20 10:05:42        27322
train/cat.10002.jpg                            2013-09-20 10:05:42        25723
train/cat.10003.jpg                            2013-09-20 10:05:44        28035
train/cat.10004.jpg                            2013-09-20 10:05:44        12973
train/cat.10005.jpg                     

In [13]:
from os import listdir
from os.path import isfile, join
import os

# loading train data from our folder in notebooks.csc.fi
mypath = join(os.getcwd(),'train')

cats_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('cat')]
dogs_imgs = [join(mypath,f) for f in listdir(mypath) if f.startswith('dog')]

In [14]:
print("cats: {} and dogs: {}".format(len(cats_imgs),len(dogs_imgs)))

cats: 12500 and dogs: 12500


In [15]:
# importing Logistic Regression, GridSearchCV, and train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings('ignore')

In [16]:
def evaluate_cat_dog(n_image=10, random_seed=42):
    start=time.time()
    Nmax = n_image
    cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
    dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]
    Y_cats = np.array(Nmax * [1])
    Y_dogs = np.array(Nmax * [0])
    X_cvd = np.vstack([cats_features,dogs_features])
    Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()
    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=random_seed)
    lg = LogisticRegression()
    grid_values = {'C': [0.001,0.1,1,10,1000]}
    grid = GridSearchCV(lg, param_grid=grid_values, cv=3).fit(X_train, y_train)
    end=time.time()
    print(f"Time for {n_image} images: ", str(end-start))
    print("Test set score: {:.2f}".format(grid.score(X_test, y_test))+'\n')

In [17]:
evaluate_cat_dog(n_image=10)

Time for 10 images:  0.971743106842041
Test set score: 1.00



In [18]:
evaluate_cat_dog(n_image=100)

Time for 100 images:  4.750216722488403
Test set score: 0.88



In [19]:
evaluate_cat_dog(n_image=500)

Time for 500 images:  21.696927309036255
Test set score: 0.93



In [20]:
evaluate_cat_dog(n_image=1000)

Time for 1000 images:  41.49506139755249
Test set score: 0.95



In [21]:
evaluate_cat_dog(n_image=5000)

Time for 5000 images:  201.74484038352966
Test set score: 0.96



In [22]:
evaluate_cat_dog(n_image=12500)

Time for 12500 images:  503.01924228668213
Test set score: 0.97



#### Task 2 (5 points):

Repeat all previous steps using MobileNet V2 (https://github.com/KeyKy/mobilenet-mxnet). How the two networks compare in terms of accuracy and running time?

**WARNING**: At least for N= 5000 and 12500 it can take some time in your computer, depending of your resources. The time can largely increases depending of your chosen classifier.

In [23]:
# Loading MobileNet V2
import mxnet as mx
path='https://github.com/KeyKy/mobilenet-mxnet/'
[mx.test_utils.download(path+'raw/master/mobilenet_v2-symbol.json'),
 mx.test_utils.download(path+'raw/master/mobilenet_v2-0000.params')]

['mobilenet_v2-symbol.json', 'mobilenet_v2-0000.params']

In [24]:
with open('mobilenet_v2-symbol.json', 'r') as f:
    for n in range(10):
        s = f.readline().strip()
        if s: print(s)

{
"nodes": [
{
"op": "null",
"name": "data",
"inputs": []
},
{
"op": "null",
"name": "conv1_weight",


In [25]:
sym, arg_params, aux_params = mx.model.load_checkpoint('mobilenet_v2', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

In [26]:
# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['conv6_4_bn_moving_var',
 'conv6_4_bn_output',
 'relu6_4_output',
 'pool6_output',
 'fc7_weight',
 'fc7_bias',
 'fc7_output',
 'fc7_output',
 'prob_label',
 'prob_output']

In [27]:
fe_sym = all_layers[422]
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [28]:
img = get_image('https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12234558/Chinook-On-White-03.jpg')
fe_mod.forward(Batch([mx.nd.array(img)]))
features = fe_mod.get_outputs()[0].asnumpy()
print(features)
assert features.shape == (1, 1000)

[[-2.53242432e+02  7.39488281e+02 -9.45433105e+02 -8.95362610e+02
  -1.60666956e+03  3.75616272e+02 -8.43659180e+02  2.19088333e+02
   8.81194885e+02 -3.07917694e+02  9.00784912e+01 -2.90159210e+02
  -1.07945972e+03 -1.68475281e+02 -6.06903931e+02  3.18268219e+02
   6.31286987e+02 -8.58956177e+02  2.75827618e+01 -1.45581470e+03
  -5.79044922e+02  1.01464929e+03  1.33638962e+02  1.34981458e+03
   7.63930664e+02 -2.00952332e+02 -1.03368042e+03 -5.26153137e+02
  -2.05386429e+02 -1.64269019e+03 -9.28135437e+02 -6.28251953e+02
  -9.85136475e+02 -1.66888391e+03 -6.00437805e+02 -9.61132568e+02
  -3.93855652e+02 -3.66343536e+02 -6.68354614e+02 -3.34592163e+02
  -8.26844849e+02 -1.20622083e+03 -5.80023315e+02  1.87059860e+02
  -3.56228394e+02 -1.10537524e+03 -8.43455566e+02 -1.39050977e+03
  -3.00516235e+02 -1.09723071e+03 -1.95988879e+03  1.62864441e+03
  -1.21257520e+03 -8.12931152e+02 -1.26317297e+03 -1.80114258e+03
  -3.32396271e+02 -7.20912720e+02 -1.84693347e+03 -9.33904053e+02
  -2.26776

In [29]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

def evaluate_cat_dog_mobilenet(n_image=10, random_seed=42):
    start=time.time()
    Nmax = n_image
    cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
    dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]
    Y_cats = np.array(Nmax * [1])
    Y_dogs = np.array(Nmax * [0])
    X_cvd = np.vstack([cats_features,dogs_features])
    Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()
    X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=random_seed)
    lg = LogisticRegression()
    grid_values = {'C': [0.001,0.1,1,10,1000]}
    grid = GridSearchCV(lg, param_grid=grid_values, cv=3).fit(X_train, y_train)
    end=time.time()
    print(f"Time for {n_image} images: ", str(end-start))
    print("Test set score: {:.2f}".format(grid.score(X_test, y_test))+'\n')

In [30]:
for n_image in [10, 100, 500, 1000, 5000, 12500]:
    evaluate_cat_dog_mobilenet(n_image=n_image)

Time for 10 images:  2.394000291824341
Test set score: 0.40

Time for 100 images:  16.524248361587524
Test set score: 0.56

Time for 500 images:  78.87917375564575
Test set score: 0.64

Time for 1000 images:  147.36745643615723
Test set score: 0.65

Time for 5000 images:  751.4943358898163
Test set score: 0.66

Time for 12500 images:  1896.7235307693481
Test set score: 0.64



In [32]:
print ('There is a significant difference between SqueezeNet_v1.2 and mobilenet_v2 and SqueezeNet_v1.2 is faster and more accurate.'),
print ('12500 image SqueezeNet_v1.2 accuracy = .97  and run time = 503 seconds'),
print ('12500 image mobilenet_v2 accuracy = .64 and run time = 1896 seconds')

There is a significant difference between SqueezeNet_v1.2 and mobilenet_v2 and SqueezeNet_v1.2 is faster and more accurate.
12500 image SqueezeNet_v1.2 accuracy = .97  and run time = 503 seconds
12500 image mobilenet_v2 accuracy = .64 and run time = 1896 seconds


In [122]:
# this result come from squeeze net model 

features_out = get_features(get_image(dog_test_path))

prob = grid.predict_proba(features_out)
pred = grid.predict(features_out)

if pred[0] == 1:
    fpred = 'cat'
else:
    fpred = 'dog'
    
print("prob: {} and prediction: {}".format(prob,fpred))

prob: [[0.96507447 0.03492553]] and prediction: dog


#### Task 3 (5 points):

Using the best network. Train a machine learning model able to predic COVID-19 from chest X-Ray images. Use the data from https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset. 

Present and discuss your best model.

In [36]:
import pandas as pd
xray = pd.read_csv('Chest_xray_Corona_Metadata.csv')
xray.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [37]:
xray[xray.Label_2_Virus_category == "COVID-19"].X_ray_image_name

5221                         kjr-21-e25-g001-l-a.jpg
5222                         kjr-21-e24-g003-l-a.jpg
5223                         kjr-21-e24-g002-l-a.jpg
5224                         kjr-21-e24-g001-l-a.jpg
5225                        jkms-35-e79-g001-l-c.jpg
5226                        jkms-35-e79-g001-l-b.jpg
5227                        jkms-35-e79-g001-l-a.jpg
5228                                   gr1_lrg-b.jpg
5229                                   gr1_lrg-a.jpg
5230                       all14238-fig-0001-m-b.jpg
5237       FE9F9A5D-2830-46F9-851B-1FF4534959BE.jpeg
5238       F63AB6CE-1968-4154-A70F-913AF154F53D.jpeg
5239       F4341CE7-73C9-45C6-99C8-8567A5484B63.jpeg
5240       F2DE909F-E19C-4900-92F5-8F435B031AC6.jpeg
5242       E63574A7-4188-4C8D-8D17-9D67A18A1AFA.jpeg
5243       E1724330-1866-4581-8CD8-CEC9B8AFEDDE.jpeg
5244       D7AF463C-2369-492D-908D-BE1911CCD74C.jpeg
5245       D5ACAA93-C779-4E22-ADFA-6A220489F840.jpeg
5246       9C34AF49-E589-44D5-92D3-168B3B04E4A

In [39]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "Chest_xray_Corona_Metadata_train.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
Chest_xray_Corona_Metadata_train/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg 2020-03-20 01:27:54       367441
Chest_xray_Corona_Metadata_train/1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg 2020-03-20 01:27:54       309985
Chest_xray_Corona_Metadata_train/171CB377-62FF-4B76-906C-F3787A01CB2E.jpeg 2020-03-20 01:27:54       586108
Chest_xray_Corona_Metadata_train/191F3B3A-2879-4EF3-BE56-EE0D2B5AAEE3.jpeg 2020-03-20 01:27:54       832615
Chest_xray_Corona_Metadata_train/1B734A89-A1BF-49A8-A1D3-66FAFA4FAC5D.jpeg 2020-03-20 01:27:54       276007
Chest_xray_Corona_Metadata_train/1-s2.0-S0140673620303706-fx1_lrg.jpg 2020-03-20 01:27:54       813392
Chest_xray_Corona_Metadata_train/1-s2.0-S0929664620300449-gr2_lrg-a.jpg 2020-03-20 01:27:54       198740
Chest_xray_Corona_Metadata_train/1-s2.0-S0929664620300449-gr2_lrg-b.jpg 2020-03-20 01:27:54       276425
Chest_xray_Corona_Metadata_train/1-s2.0-S0929664620300449-gr2_lrg-c

In [148]:
xray_path = join(os.getcwd(), 'train')
covid = [join(xray_path, f) for f in xray[xray.Label_2_Virus_category == "COVID-19"].X_ray_image_name]
normal = [join(xray_path, f) for f in listdir(xray_path) if f not in xray[xray.Label_2_Virus_category == "COVID-19"].X_ray_image_name]

In [149]:
print(f"covid n = {len(covid)} and normal image n = {len(normal)}")

covid n = 58 and normal image n = 5309


In [135]:
sym, arg_params, aux_params = mx.model.load_checkpoint('model', 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [136]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [175]:
def evaluate_covid(random_seed=42):
    start=time.time()
    covid_features = [get_features(get_image(img)).ravel() for img in covid]
    normal_features = [get_features(get_image(img)).ravel() for img in normal]
    Y_covid = np.array(len(covid) * [1])
    Y_normal = np.array(len(normal) * [0])
    X_cvn = np.vstack([covid_features,normal_features])
    Y_cvn = np.append(np.ones(len(covid)) , np.zeros(len(normal))).ravel()
    X_train, X_test, y_train, y_test = train_test_split(X_cvn, Y_cvn, random_state=random_seed, test_size=.4)
    lg = LogisticRegression()
    grid_values = {'C': [0.001,0.1,1,10,1000]}
    grid = GridSearchCV(lg, param_grid=grid_values, cv=3).fit(X_train, y_train)
    end=time.time()
    print(f"Time for {n_image} image: ", str(end-start))
    print("Test set score: {:.2f}".format(grid.score(X_test, y_test))+'\n')

In [176]:
evaluate_covid()

Time for 12500 image:  173.87865781784058
Test set score: 0.99



In [181]:
from sklearn.metrics import classification_report

# Build a text report showing the main classification metrics
print(classification_report(grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         3

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

